In [10]:
import os
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re
from lxml import etree 
import json

# www.englishclub.com/esl-quizzes

## Test

In [11]:
URL = 'https://www.englishclub.com/vocabulary/animal-terms-easy-quiz.php'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [12]:
quiz_name = soup.find("h1").text
questions = soup.find_all("form", class_="mcquiz")[0]
choices = questions.find_all("input")

In [13]:
# extract answer
answer_str = soup.find_all('script', {"type": "text/javascript", "language":"JavaScript"})[0].text
pattern = r'answers\[\d\]="([^"]+)"'
matches = re.findall(pattern, answer_str)

list_qno = []
list_answers = []
count_q = 1
for match in matches:
    list_answers.append(match)
    list_qno.append(count_q)
    count_q = count_q+1
df_answers = pd.DataFrame({'question_no':list_qno, 'answer':list_answers})
# df_answers

In [14]:
list_qno = []
list_questions = []
count_q = 1
for q in questions.find_all("p")[:len(list_answers)]:
    pattern = r'^\d+\.\s*'
    str_wo_num = re.sub(pattern, '', q.text)
    list_questions.append(str_wo_num)
    list_qno.append(count_q)
    count_q = count_q+1
    
df_questions = pd.DataFrame({'question_no':list_qno, 'question':list_questions})
# df_questions

In [15]:
list_choices_qno = []
list_choices = []
list_choices_no = []
for i in range(1, len(list_answers)+1):
   
    count_c = 1
    for c in choices[:-3]:
#         print(c)
        try:
            if int(c.get('name')[1:])==i:
                list_choices_qno.append(i)
                list_choices_no.append(count_c)
                list_choices.append(c.get('value'))
                count_c = count_c+1
        except:
            pass


In [16]:
df_choice = pd.DataFrame({'question_no':list_choices_qno, 'choices_no':list_choices_no, 'choices':list_choices})

In [17]:
# df_choice.head()

In [18]:
df_quiz = pd.merge(df_questions, df_answers, on='question_no')
df_quiz = df_quiz.merge(df_choice, on='question_no')
# df_quiz.shape
df_quiz.head()

,question_no,question,answer,choices_no,choices
0,1,A male chicken is called,a rooster,1,a hen
1,1,A male chicken is called,a rooster,2,a chick
2,1,A male chicken is called,a rooster,3,a rooster
3,2,A young sheep is called,a lamb,1,a ram
4,2,A young sheep is called,a lamb,2,a lamb


In [23]:
dict_q = {'quizname': quiz_name}
dict_q['description'] = 'This is the a sample quiz'
dict_q['category'] = 'English'
dict_q['questions'] = []
for i in df_quiz['question_no'].unique():
    dict_qa = {}
    question_info = df_quiz[df_quiz['question_no']==i]
    question = question_info['question'].unique()[0]
    answer = question_info['answer'].unique()[0]
    choices = {}
    for c in question_info['choices_no'].unique():
        choices[str(c)] = df_quiz[(df_quiz['question_no']==i)&(df_quiz['choices_no']==c)]['choices'].values[0]

    dict_qa = {'type': 'mc',
               'points': '1',
               'text': question, 
               'answer': answer,
               'choices': choices}

    dict_q['questions'].append(dict_qa)

{'name': 'Animal Terms (short list) Quiz',
 'questions': [{'type': 'mc',
   'points': '1',
   'text': 'A male chicken is called',
   'answer': 'a rooster',
   'choices': {'1': 'a hen', '2': 'a chick', '3': 'a rooster'}},
  {'type': 'mc',
   'points': '1',
   'text': 'A young sheep is called ',
   'answer': 'a lamb',
   'choices': {'1': 'a ram', '2': 'a lamb', '3': 'a ewe'}},
  {'type': 'mc',
   'points': '1',
   'text': 'A female pig is called ',
   'answer': 'a sow',
   'choices': {'1': 'a sow', '2': 'a suckling', '3': 'a boar'}},
  {'type': 'mc',
   'points': '1',
   'text': 'A baby kangaroo is called',
   'answer': 'a joey',
   'choices': {'1': 'a cub', '2': 'a pup', '3': 'a joey'}},
  {'type': 'mc',
   'points': '1',
   'text': 'A tigress is a _______ tiger.',
   'answer': 'female',
   'choices': {'1': 'male', '2': 'female', '3': 'young'}},
  {'type': 'mc',
   'points': '1',
   'text': 'A foal is a _______ horse. ',
   'answer': 'young',
   'choices': {'1': 'male', '2': 'female', '

In [20]:
# yok's version
# dict_q = {'qname': quiz_name}
# dict_q['qtype'] = 'choice_multiple'

# dict_q['questions'] = {}
# for i in df_quiz['question_no'].unique():
#     dict_qa = {}
#     question_info = df_quiz[df_quiz['question_no']==i]
#     question = question_info['question'].unique()[0]
#     answer = question_info['answer'].unique()[0]
#     choices = {}
#     for c in question_info['choices_no'].unique():
#         choices[str(c)] = df_quiz[(df_quiz['question_no']==i)&(df_quiz['choices_no']==c)]['choices'].values[0]
        
#     dict_qa = {'question': question, 
#                'answer': answer,
#                'choices': choices}
    
#     dict_q['questions'][str(i)] = dict_qa

# dict_q

{'qname': 'Animal Terms (short list) Quiz',
 'qtype': 'choice_multiple',
 'questions': {'1': {'question': 'A male chicken is called',
   'answer': 'a rooster',
   'choices': {'1': 'a hen', '2': 'a chick', '3': 'a rooster'}},
  '2': {'question': 'A young sheep is called ',
   'answer': 'a lamb',
   'choices': {'1': 'a ram', '2': 'a lamb', '3': 'a ewe'}},
  '3': {'question': 'A female pig is called ',
   'answer': 'a sow',
   'choices': {'1': 'a sow', '2': 'a suckling', '3': 'a boar'}},
  '4': {'question': 'A baby kangaroo is called',
   'answer': 'a joey',
   'choices': {'1': 'a cub', '2': 'a pup', '3': 'a joey'}},
  '5': {'question': 'A tigress is a _______ tiger.',
   'answer': 'female',
   'choices': {'1': 'male', '2': 'female', '3': 'young'}},
  '6': {'question': 'A foal is a _______ horse. ',
   'answer': 'young',
   'choices': {'1': 'male', '2': 'female', '3': 'young'}},
  '7': {'question': 'A vixen is a _______ fox.',
   'answer': 'female',
   'choices': {'1': 'male', '2': 'femal

In [138]:
path_cur = os.getcwd()
path_mainp = "/".join(path_cur.split('/')[:-1])
path_dbquizzes = os.path.join(path_mainp, 'db_quizzes')

path_qname = URL.split('/')[-1][:-4]
path_dbquizzes_filename = os.path.join(path_dbquizzes, path_qname)+'.json'
# path_dbquizzes_filename

In [139]:
# Convert and write JSON object to file
with open(path_dbquizzes_filename, "w") as outfile: 
    json.dump(dict_q, outfile)

### TrueFalse quiz

In [35]:
URL = 'https://www.englishclub.com/speaking/small-talk_quiz.php'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [36]:
quiz_name = soup.find("h1").text
questions = soup.find_all("form")[0]
choices = questions.find_all("input")

In [37]:
# extract answer
answer_str = soup.find_all('script', {"type": "text/javascript", "language":"JavaScript"})[0].text
pattern = r'\b(True|False)\b'
matches = re.findall(pattern, answer_str)

list_qno = []
list_answers = []
count_q = 1
for match in matches:
    list_answers.append(match)
    list_qno.append(count_q)
    count_q = count_q+1
df_answers = pd.DataFrame({'question_no':list_qno, 'answer':list_answers})
# df_answers

In [38]:
list_qno = []
list_questions = []
count_q = 1
for q in questions.find_all("p")[:len(list_answers)]:
    pattern = r'^\d+\.\s*'
    str_wo_num = re.sub(pattern, '', q.text.split('\n')[0])
    list_questions.append(str_wo_num)
    list_qno.append(count_q)
    count_q = count_q+1
    
df_questions = pd.DataFrame({'question_no':list_qno, 'question':list_questions})
# df_questions

In [39]:
list_choices_qno = []
list_choices = []
list_choices_no = []
for i in range(1, len(list_answers)+1):
   
    count_c = 1
    for c in choices[:-3]:
#         print(c)
        try:
            if int(c.get('name')[1:])==i:
                list_choices_qno.append(i)
                list_choices_no.append(count_c)
                list_choices.append(c.get('value'))
                count_c = count_c+1
        except:
            pass


In [40]:
df_choice = pd.DataFrame({'question_no':list_choices_qno, 'choices_no':list_choices_no, 'choices':list_choices})

In [41]:
df_quiz = pd.merge(df_questions, df_answers, on='question_no')
df_quiz = df_quiz.merge(df_choice, on='question_no')
print(df_quiz.shape)
# df_quiz.head()

(20, 5)


In [42]:
dict_q = {'quizname': quiz_name}
dict_q['description'] = 'This is the a sample quiz'
dict_q['category'] = 'English'
dict_q['questions'] = []
for i in df_quiz['question_no'].unique():
    dict_qa = {}
    question_info = df_quiz[df_quiz['question_no']==i]
    question = question_info['question'].unique()[0]
    answer = question_info['answer'].unique()[0]
    choices = {}
    for c in question_info['choices_no'].unique():
        choices[str(c)] = df_quiz[(df_quiz['question_no']==i)&(df_quiz['choices_no']==c)]['choices'].values[0]

    dict_qa = {'type': 'tf',
               'points': '1',
               'text': question, 
               'answer': answer,
               'choices': choices}

    dict_q['questions'].append(dict_qa)

In [43]:
dict_q

{'quizname': 'Small Talk Quiz',
 'description': 'This is the a sample quiz',
 'category': 'English',
 'questions': [{'type': 'tf',
   'points': '1',
   'text': 'It is common for people who are waiting together to engage in small talk.',
   'answer': 'True',
   'choices': {'1': 'True', '2': 'False'}},
  {'type': 'tf',
   'points': '1',
   'text': 'Religion is usually considered a "safe" topic for small talk.',
   'answer': 'False',
   'choices': {'1': 'True', '2': 'False'}},
  {'type': 'tf',
   'points': '1',
   'text': 'Children are taught not to speak to strangers who engage them in small talk.',
   'answer': 'True',
   'choices': {'1': 'True', '2': 'False'}},
  {'type': 'tf',
   'points': '1',
   'text': "It is rude to make small talk with someone who's delivering your mail.",
   'answer': 'False',
   'choices': {'1': 'True', '2': 'False'}},
  {'type': 'tf',
   'points': '1',
   'text': 'Sport is not usually a "safe" topic for small talk.',
   'answer': 'False',
   'choices': {'1': '

In [44]:
path_cur = os.getcwd()
path_mainp = "/".join(path_cur.split('/')[:-1])
path_dbquizzes = os.path.join(path_mainp, 'db_quizzes')

path_qname = "english-"+"-".join(URL.split('/')[-2:])[:-4]
path_dbquizzes_filename = os.path.join(path_dbquizzes, path_qname)+'.json'
# path_dbquizzes_filename

# Convert and write JSON object to file
with open(path_dbquizzes_filename, "w") as outfile: 
    json.dump(dict_q, outfile)

## Auto scrape multiple choices

In [45]:
import os
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re
from lxml import etree 
import json

In [47]:
def auto_scrape(URL):
#     URL = 'https://www.englishclub.com/vocabulary/animal-terms-easy-quiz.php'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    quiz_name = soup.find("h1").text
    questions = soup.find_all("form", class_="mcquiz")[0]
    choices = questions.find_all("input")

    # extract answer
    answer_str = soup.find_all('script', {"type": "text/javascript", "language":"JavaScript"})[0].text
    pattern = r'answers\[\d\]="([^"]+)"'
    matches = re.findall(pattern, answer_str)

    list_qno = []
    list_answers = []
    count_q = 1
    for match in matches:
        list_answers.append(match)
        list_qno.append(count_q)
        count_q = count_q+1
    df_answers = pd.DataFrame({'question_no':list_qno, 'answer':list_answers})
    # df_answers

    list_qno = []
    list_questions = []
    count_q = 1
    for q in questions.find_all("p")[:len(list_answers)]:
        pattern = r'^\d+\.\s*'
        str_wo_num = re.sub(pattern, '', q.text)
        list_questions.append(str_wo_num)
        list_qno.append(count_q)
        count_q = count_q+1

    df_questions = pd.DataFrame({'question_no':list_qno, 'question':list_questions})
    # df_questions

    list_choices_qno = []
    list_choices = []
    list_choices_no = []
    for i in range(1, len(list_answers)+1):

        count_c = 1
        for c in choices[:-3]:
    #         print(c)
            try:
                if int(c.get('name')[1:])==i:
                    list_choices_qno.append(i)
                    list_choices_no.append(count_c)
                    list_choices.append(c.get('value'))
                    count_c = count_c+1
            except:
                pass


    df_choice = pd.DataFrame({'question_no':list_choices_qno, 'choices_no':list_choices_no, 'choices':list_choices})

    # df_choice.head()

    df_quiz = pd.merge(df_questions, df_answers, on='question_no')
    df_quiz = df_quiz.merge(df_choice, on='question_no')
    # df_quiz.shape
    # df_quiz.head()

    dict_q = {'quizname': quiz_name}
    dict_q['description'] = 'This is the a sample quiz'
    dict_q['category'] = 'English'
    dict_q['questions'] = []
    for i in df_quiz['question_no'].unique():
        dict_qa = {}
        question_info = df_quiz[df_quiz['question_no']==i]
        question = question_info['question'].unique()[0]
        answer = question_info['answer'].unique()[0]
        choices = {}
        for c in question_info['choices_no'].unique():
            choices[str(c)] = df_quiz[(df_quiz['question_no']==i)&(df_quiz['choices_no']==c)]['choices'].values[0]

        dict_qa = {'type': 'tf',
                   'points': '1',
                   'text': question, 
                   'answer': answer,
                   'choices': choices}

        dict_q['questions'].append(dict_qa)

    # dict_q

    path_cur = os.getcwd()
    path_mainp = "/".join(path_cur.split('/')[:-1])
    path_dbquizzes = os.path.join(path_mainp, 'db_quizzes')

    path_qname = "english-" + "-".join(URL.split('/')[-2:])[:-4]
    path_dbquizzes_filename = os.path.join(path_dbquizzes, path_qname)+'.json'
    # path_dbquizzes_filename

    # Convert and write JSON object to file
    with open(path_dbquizzes_filename, "w") as outfile: 
        json.dump(dict_q, outfile, indent = 4)

In [48]:
# https://www.englishclub.com/esl-quizzes/vocabulary/
# Mixed Topics

list_url = [
'https://www.englishclub.com/vocabulary/animal-terms-quiz.php',
'https://www.englishclub.com/vocabulary/festivals-christmas-quiz.php',
'https://www.englishclub.com/vocabulary/computer-quiz.php',
'https://www.englishclub.com/vocabulary/world-countries-nationality-quiz.php',
'https://www.englishclub.com/vocabulary/criminals-quiz.php',
'https://www.englishclub.com/vocabulary/family-quiz.php',
'https://www.englishclub.com/vocabulary/nsew-quiz.php',
'https://www.englishclub.com/vocabulary/numbers-quiz.php',
# 'https://www.englishclub.com/speaking/small-talk_quiz.php',  # This is the True False quiz
'https://www.englishclub.com/vocabulary/collocations-quiz-time.php',
'https://www.englishclub.com/vocabulary/time-quiz.php',
'https://www.englishclub.com/speaking/weather-vocabulary-quiz.php',
'https://www.englishclub.com/vocabulary/weather-vocabulary-quiz.php',
'https://www.englishclub.com/vocabulary/festivals-christmas-quiz.php'  
]

for iurl in list_url:
    auto_scrape(iurl)